In [1]:
import torch
torch.cuda.is_available()

False

In [8]:
import pandas as pd
df = pd.read_csv('/Users/suchirnaik/Downloads/three_hundred_labeled_samples_suchir.csv')


In [9]:
df

,parent_comment,comment,label
0,"If you ever go to any of the ""Crossroads of th...","Thanks, I've been meaning to go to a gun show,...",0
1,All 3 services have the medic role you are tal...,Aged care placement and care plans are integra...,0
2,We were too nice to the south after the war du...,"yeah, I'm sure continuing the scorched-earth, ...",1
3,If this offer existed...,Now **that's** a *great* idea!,0
4,So I guess this means that even solar power is...,Those birds would have died of thirst anyways.,1
...,...,...,...
268,Trump would have an amazing legacy in my mind ...,Yeah no scientist has ever or will ever be wro...,1
269,"If you ask my dad, me apparently",:( happy holidays,0
270,"""Secretary of Health and Human Services sounds...","I mean, how wide is the scope of the president?",1
271,Pounds,So $100 bucks ;),0


In [10]:
df['label'].value_counts()

0    147
1    126
Name: label, dtype: int64

In [11]:
from torch.utils.data import TensorDataset, DataLoader

In [12]:
df['combined_text'] = df['comment'] + ' ' + df['parent_comment']

In [16]:
from transformers import BertTokenizer
# Tokenize the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_data = tokenizer(df['combined_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

In [18]:
# Assuming df is your complete DataFrame
labels_complete = torch.tensor(df['label'].tolist())


In [23]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded_data['input_ids'], labels_complete , test_size=0.2, random_state=42)

In [25]:
# Define a DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [26]:
from transformers import BertForSequenceClassification
# Load the pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
from torch.utils.data import TensorDataset, DataLoader

In [27]:
# Define the loss function and optimizer
import torch.nn as nn 
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

In [28]:
from tqdm.notebook import tqdm  # Import tqdm for progress bar

# Training loop
model.train()
total_epochs = 5  

for epoch in range(total_epochs):
    print(f"Epoch {epoch + 1}/{total_epochs}")
    
    # Use tqdm for a progress bar over the batches
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, labels = batch
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Epoch 1/3


Epoch 1:   0%|          | 0/14 [00:00<?, ?it/s]

Epoch 2/3


Epoch 2:   0%|          | 0/14 [00:00<?, ?it/s]

Epoch 3/3


Epoch 3:   0%|          | 0/14 [00:00<?, ?it/s]

In [29]:
model.eval()
with torch.no_grad():
    outputs = model(test_inputs, labels=test_labels)
    predicted_labels = torch.argmax(outputs.logits, dim=1)

accuracy = (predicted_labels == test_labels).sum().item() / len(test_labels)
print('Accuracy:', accuracy)


Accuracy: 0.45454545454545453
